In [1]:
# import libraries and navigate to directory
%cd '/home/uzumochi/eigenjuno/data/raw/perijove_9'
from stitch import *
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os, threading, time, multiprocessing, logging, sys, json, math

/home/uzumochi/eigenjuno/data/raw/perijove_9


In [2]:
# handle frequent spice exceptions - code still runs fine
f_err = sys.stderr = open('errors.txt', 'w')
get_ipython().log.handlers[0].stream = f_err
get_ipython().log.setLevel(logging.INFO)

In [5]:
# stitch function for thread, taken largely from the reddit post linked in repo desc
def stitch(meta_num):
    im_info = meta_num + '-Metadata.json'
    with open(im_info, 'r') as json_file:
        im_info_dir = json.load(json_file)
        image = im_info_dir['FILE_NAME']
        image_time = im_info_dir['IMAGE_TIME']
    
    img = Image.open(image)
    im_ar = np.array(img)
    
    s1, s2 = im_ar.shape

    start_time = im_info_dir["START_TIME"]
    frame_delay = float(im_info_dir["INTERFRAME_DELAY"].split()[0])+0.001
    
    start_correction, frame_delay = correct_image_start_time_and_frame_delay(im_ar, start_time, frame_delay)
    
    framelets = generate_framelets(im_ar, start_time, start_correction, frame_delay)
    # removed color correction - im_ar.astype(np.int) ** 2

    cam_pos, cam_orient = get_junocam_jupiter_rel_pos_orient(start_time, start_correction + 17 * frame_delay)

    y, x = np.mgrid[-1024:1024,-1024:1024]
    x += 300
    y += 50
    rays = np.concatenate([x[...,None], y[...,None], np.ones(((2048),(2048),1))*fl[0]], axis=-1)
    rays = rays.dot(cam_orient)

    surface_raster, _ = project_onto_jupiter_surf(cam_pos, rays)
    
#     with open('surf_points.txt') as f:
#         f.write(surface_raster)

    colors = np.zeros(((2048),(2048),3))
    color_counts = np.zeros(((2048),(2048),3))

    for k,framelet in enumerate(framelets):
        print(str(meta_num) + ': processing framelet {} of {}..'.format(k+1, len(framelets)))
        col = framelet.color
        brightnesses, valid_map = framelet.get_pixel_val_at_surf_point(surface_raster) # this is where it slows down
        colors[...,2-col] += brightnesses
        color_counts[...,2-col] += valid_map

    colors /= np.maximum(color_counts, 1)
    colors *= 255 / np.max(colors)

    colors = colors.astype(np.uint8)

    new_img = Image.fromarray(colors)
    new_img.save("out_" + str(meta_num) + ".png")

In [6]:
#begin multithreaded execution - this will process all images in a directory, assuming they are from the same perijove
img_nums = []
for files in os.listdir('/home/uzumochi/eigenjuno/data/raw/perijove_9'):
    if files.endswith('.json') and not os.path.isfile('out_' + files[0 : 4] + '.png'):
        img_nums.append(files[0 : 4])
print_str = 'Processing the following images: '
for im in img_nums:
    print_str += (im + ', ')
print_str = print_str[:-2]
print(print_str)

num_cpus = multiprocessing.cpu_count() - 1
curr = 0

while curr < len(img_nums):
    threads = []
    num_threads = num_cpus if num_cpus < len(img_nums) else len(img_nums)
    print('Running ' + str(num_threads) + ' threads')
    for t in range(num_threads):
        try:
            threads.append(threading.Thread(target = stitch, args = (str(img_nums[curr]), )))
        except IndexError:
            print('\nStarting new round of threads')
        threads[t].start()
        curr += 1
    for t in threads:
        t.join()

print("done! :D")

Processing the following images: 3094, 3100
Running 2 threads
3100: processing framelet 1 of 102..
3094: processing framelet 1 of 126..
done! :D
